# Visualization of Stock Trading

In [1]:
# import libraries
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns

## Set relevant parameters

In [2]:
train_stock = 'data/GOOGL.csv'
val_stock = 'data/BTC-USD_Val.csv'
window_size = 10
batch_size = 16
ep_count = 10
model_name = 'model_GOOGL'
pretrained = False
debug = False

## Load Dataset into pandas DataFrame

In [4]:
# read csv into dataframe
df = pd.read_csv(val_stock)

# filter out the desired features
df = df[['Date', 'Adj Close']]

# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})

df.head()

,date,actual
0,2018-12-10,3502.656006
1,2018-12-11,3424.588135
2,2018-12-12,3486.950195
3,2018-12-13,3313.677246
4,2018-12-14,3242.484863


In [5]:
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

In [10]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
date      366 non-null datetime64[ns]
actual    366 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.8 KB


,date,actual
0,2018-12-10,3502.656006
1,2018-12-11,3424.588135
2,2018-12-12,3486.950195
3,2018-12-13,3313.677246
4,2018-12-14,3242.484863


## Visualization of actions taken by Trading Bot

In [6]:
def visualize(df, history):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
        #color=alt.Color('action', scale=alt.Scale(range=['blue', 'red']))
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=val_stock).properties(height=300, width=1000)
    
    return chart

## Training the model

In [9]:
import logging
import coloredlogs

from train import *
from evaluate import show_eval_result

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

agent = Agent(window_size, pretrained=pretrained, model_name=model_name)
train_data = get_stock_data(train_stock)
val_data = get_stock_data(val_stock)

initial_offset = val_data[1] - val_data[0]

for i in range(1, ep_count + 1):
    train_result = train_model(agent, i, train_data, ep_count=ep_count,
                                batch_size=batch_size, window_size=window_size)
    val_result, _ = evaluate_model(agent, val_data, window_size, debug)
    show_train_result(train_result, val_result, initial_offset)

Using TensorFlow backend.
2019-12-26 10:52:03 MR-ROBOT root[19500] DEBUG switching to TensorFlow for CPU
Episode 1/10: 100%|█████████████████████████████████████████████████████████████| 1760/1760 [01:08<00:00, 25.71it/s]
2019-12-26 10:53:12 MR-ROBOT root[19500] INFO Episode 1/10 - Train Position: +$1173.44  Val Position: -$5628.36  Train Loss: 0.6349  (~68.4679 secs)
Episode 2/10: 100%|█████████████████████████████████████████████████████████████| 1760/1760 [01:08<00:00, 25.86it/s]
2019-12-26 10:54:21 MR-ROBOT root[19500] INFO Episode 2/10 - Train Position: +$176.23  Val Position: +$2648.37  Train Loss: 8.2730  (~68.0636 secs)
Episode 3/10: 100%|█████████████████████████████████████████████████████████████| 1760/1760 [01:07<00:00, 26.05it/s]
2019-12-26 10:55:29 MR-ROBOT root[19500] INFO Episode 3/10 - Train Position: +$526.33  Val Position: +$37104.51  Train Loss: 2.6399  (~67.5699 secs)
Episode 4/10: 100%|█████████████████████████████████████████████████████████████| 1760/1760 [01:07

## Plotting the evaluation actions

In [10]:
val_result, history = evaluate_model(agent, val_data, window_size, debug)
show_eval_result(model_name, val_result, initial_offset)
chart = visualize(df, history)
chart

2019-12-26 11:30:03 MR-ROBOT root[19500] INFO model_GOOGL: +$5879.64

2019-12-26 11:30:03 MR-ROBOT numexpr.utils[19500] INFO NumExpr defaulting to 8 threads.


alt.LayerChart(...)